# Extract Custom Fields from Your File

This notebook demonstrates how to use analyzers to extract custom fields from your input files.

## Prerequisites
1. Please ensure your Azure AI service is configured by following the [configuration steps](../README.md#configure-azure-ai-service-resource).
2. Install the required packages to run this sample.

In [ ]:
%pip install -r ../requirements.txt

## Create Azure AI Content Understanding Client

> The [AzureContentUnderstandingClient](../python/content_understanding_client.py) is a utility class providing functions to interact with the Content Understanding API. Before the official release of the Content Understanding SDK, it acts as a lightweight SDK. Please fill in the constants **AZURE_AI_ENDPOINT** and **AZURE_AI_API_VERSION** with your Azure AI Service information. Optionally, you may provide **AZURE_AI_API_KEY** if your setup requires key-based authentication.

> ⚠️ Important:
Please update the code below to match your Azure authentication method.
Look for the `# IMPORTANT` comments and modify those sections accordingly.
Skipping this step may cause the sample to not run correctly.

> ⚠️ Note: Using a subscription key works, but using a token provider with Azure Active Directory (AAD) is safer and highly recommended for production environments.

In [ ]:
import logging
import json
import os
import sys
import uuid
from pathlib import Path
from dotenv import find_dotenv, load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

load_dotenv(find_dotenv())
logging.basicConfig(level=logging.INFO)

# For authentication, you can use either token-based authentication or a subscription key; only one method is required.
AZURE_AI_ENDPOINT = os.getenv("AZURE_AI_ENDPOINT")
# IMPORTANT: Replace with your actual subscription key or set it in the ".env" file if not using token authentication.
AZURE_AI_API_KEY = os.getenv("AZURE_AI_API_KEY")
AZURE_AI_API_VERSION = os.getenv("AZURE_AI_API_VERSION", "2025-05-01-preview")

# Add the parent directory to the path to use shared modules
parent_dir = Path(Path.cwd()).parent
sys.path.append(str(parent_dir))
from python.content_understanding_client import AzureContentUnderstandingClient

credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

client = AzureContentUnderstandingClient(
    endpoint=AZURE_AI_ENDPOINT,
    api_version=AZURE_AI_API_VERSION,
    # IMPORTANT: Comment out token_provider if using subscription key
    token_provider=token_provider,
    # IMPORTANT: Uncomment this if using subscription key
    # subscription_key=AZURE_AI_API_KEY,
    x_ms_useragent="azure-ai-content-understanding-python/field_extraction",  # This header is used for sample usage telemetry; please comment out this line if you wish to opt out.
)

## Analyzer Templates

This notebook demonstrates field extraction across multiple modalities using Azure AI Content Understanding. We will walk through each modality step-by-step:

1. **Document Analysis** - Extract fields from invoices and receipts
2. **Audio Analysis** - Process call recordings and conversational audio  
3. **Video Analysis** - Analyze marketing videos and extract insights
4. **Image Analysis** - Extract information from charts and images

Each section will create an analyzer, process sample data, display results, and clean up the analyzer before moving on to the next modality.

### Understanding Analyzer Templates and Schemas

Before creating our first analyzer, it's important to understand the structure and schema of analyzer templates. Custom analyzers in Azure AI Content Understanding are defined using JSON templates specifying which fields to extract and how to process the content.

**Key Schema Components:**

- **`baseAnalyzerId`**: Crucial setting that specifies the prebuilt analyzer to derive from (e.g., `prebuilt-documentAnalyzer`, `prebuilt-audioAnalyzer`, `prebuilt-videoAnalyzer`, `prebuilt-imageAnalyzer`, `prebuilt-callCenter`). This provides foundational capabilities for your custom analyzer.

- **`fields`**: Defines specific data points to extract. Each field includes:
  - **Field name**: Identifier for the extracted data (required and important for referencing results)
  - **Description**: Optional but helpful for documentation and understanding the field's purpose
  - **Method**: Can be `"extract"` (for extracting existing information from documents - currently only available for document analysis) or `"generate"` (for generating new insights using AI)

- **`method`**: The overall extraction approach - use `"extract"` for standard field extraction from documents or `"generate"` when AI should generate insights or summaries.

**Important Note**: The `"extract"` method is currently only available for document analysis (using `prebuilt-documentAnalyzer`). For other modalities like audio, video, and images, use the `"generate"` method.

Let's examine the invoice template to see these concepts in action:

The following schema defines the document modality analyzer used to extract fields from an invoice PDF. This analyzer identifies key invoice elements such as vendor information, amounts, dates, and line items.

In [ ]:
analyzer_template_path = '../analyzer_templates/invoice.json'
with open(analyzer_template_path, 'r') as f:
    template_content = json.load(f)
    print(json.dumps(template_content, indent=2))

Below is an example schema for the video modality.

In [ ]:
analyzer_template_path = '../analyzer_templates/marketing_video.json'
with open(analyzer_template_path, 'r') as f:
    template_content = json.load(f)
    print(json.dumps(template_content, indent=2))

# Document Analysis

Let's start with document analysis by extracting fields from invoices and receipts. This modality is ideal for processing structured documents and extracting key information such as amounts, dates, vendor details, and line items.

## 1. Invoice Field Extraction

Let's extract fields from an invoice PDF. This analyzer identifies essential invoice elements such as vendor information, amounts, dates, and line items.

In [ ]:
analyzer_template_path = '../analyzer_templates/invoice.json'
with open(analyzer_template_path, 'r') as f:
    template_content = json.load(f)
    print(json.dumps(template_content, indent=2))

**Create and Run Invoice Analyzer**

Now, let's create the invoice analyzer and process our sample invoice:

In [ ]:
sample_file_path = '../data/invoice.pdf'
invoice_analyzer_id = "invoice-extraction-" + str(uuid.uuid4())

print(f"Creating invoice analyzer: {invoice_analyzer_id}")
response = client.begin_create_analyzer(invoice_analyzer_id, analyzer_template_path=analyzer_template_path)
result = client.poll_result(response)
print("✅ Invoice analyzer created successfully!")

print(f"Analyzing invoice: {sample_file_path}")
response = client.begin_analyze(invoice_analyzer_id, file_location=sample_file_path)
result_json = client.poll_result(response)

**Invoice Analysis Results**

Let's examine the extracted fields from the invoice:

In [ ]:
print(json.dumps(result_json, indent=2))

**Clean Up Invoice Analyzer**

Optionally clean up the analyzer to manage resources (in production, analyzers are typically kept for reuse):

In [ ]:
client.delete_analyzer(invoice_analyzer_id)

## 2. Invoice Field Extraction with Source Grounding

Now, let's analyze the same invoice but with enhanced field source information and confidence scores. This provides additional context about the location of each extracted field within the document.

**Note**: This custom analyzer is for demonstration purposes only. In production, please start from `prebuilt-invoice` for invoice processing.

**Key Feature**: This analyzer template uses `estimateFieldSourceAndConfidence: true` in the 'config', enabling the service to provide detailed information about:
- **Field source locations**: Exact coordinates and bounding boxes indicating where each field was found
- **Confidence scores**: The confidence level of each extracted field
- **Enhanced metadata**: Additional context about the extraction process

This feature is particularly useful for applications that require verification of extraction accuracy or visual feedback indicating field locations in the source document.

In [ ]:
analyzer_template_path = '../analyzer_templates/invoice_field_source.json'
with open(analyzer_template_path, 'r') as f:
    template_content = json.load(f)
    print(json.dumps(template_content, indent=2))

**Create and Run Invoice Field Source Analyzer**

Now, let's create the enhanced analyzer and process the invoice with source grounding:

In [ ]:
sample_file_path = '../data/invoice.pdf'
invoice_source_analyzer_id = "invoice-field-source-" + str(uuid.uuid4())

print(f"Creating invoice field source analyzer: {invoice_source_analyzer_id}")
response = client.begin_create_analyzer(invoice_source_analyzer_id, analyzer_template_path=analyzer_template_path)
result = client.poll_result(response)
print("✅ Invoice field source analyzer created successfully!")

print(f"Analyzing invoice with field source: {sample_file_path}")
response = client.begin_analyze(invoice_source_analyzer_id, file_location=sample_file_path)
result_json = client.poll_result(response)

**Invoice Field Source Analysis Results**

Let's review the enhanced results, which include detailed field source locations and confidence scores. Please pay special attention to the `confidence` and `source` attributes for each extracted field. The `source` attribute provides the page number and bounding box coordinates of the extracted data, giving precise context for verification and visualization.

In [ ]:
print(json.dumps(result_json, indent=2))

**Clean Up Invoice Field Source Analyzer**

Optionally clean up the field source analyzer:

In [ ]:
client.delete_analyzer(invoice_source_analyzer_id)

## 3. Receipt Field Extraction

Let's extract information from a receipt image. This demonstrates how the Content Understanding service can handle different document types and formats.

**Receipt Analyzer Template**

Let's examine the receipt analyzer template:

In [ ]:
analyzer_template_path = '../analyzer_templates/receipt.json'
with open(analyzer_template_path, 'r') as f:
    template_content = json.load(f)
    print(json.dumps(template_content, indent=2))

**Create and Run Receipt Analyzer**

Now, let's create the receipt analyzer and process our sample receipt:

In [ ]:
sample_file_path = '../data/receipt.png'
receipt_analyzer_id = "receipt-extraction-" + str(uuid.uuid4())

print(f"Creating receipt analyzer: {receipt_analyzer_id}")
response = client.begin_create_analyzer(receipt_analyzer_id, analyzer_template_path=analyzer_template_path)
result = client.poll_result(response)
print("✅ Receipt analyzer created successfully!")

print(f"Analyzing receipt: {sample_file_path}")
response = client.begin_analyze(receipt_analyzer_id, file_location=sample_file_path)
result_json = client.poll_result(response)

**Receipt Analysis Results**

Let's examine the extracted fields from the receipt:

In [ ]:
print(json.dumps(result_json, indent=2))

**Clean Up Receipt Analyzer**

Optionally clean up the receipt analyzer:

In [ ]:
client.delete_analyzer(receipt_analyzer_id)

# Audio Analysis

Next, let's move to audio analysis. This modality allows us to extract insights from audio files such as call recordings, including summaries, sentiment analysis, topic identification, and entity extraction.

## 4. Call Recording Analytics

Let's analyze a call center recording to extract insights such as summary, topics discussed, companies mentioned, and people involved in the conversation.

**Call Recording Analytics Template**

Let's examine the call recording analytics template:

In [ ]:
analyzer_template_path = '../analyzer_templates/call_recording_analytics.json'
with open(analyzer_template_path, 'r') as f:
    template_content = json.load(f)
    print(json.dumps(template_content, indent=2))

**Create and Run Call Recording Analyzer**

In [ ]:
sample_file_path = '../data/callCenterRecording.mp3'
call_analyzer_id = "call-recording-analytics-" + str(uuid.uuid4())

print(f"Creating call recording analyzer: {call_analyzer_id}")
response = client.begin_create_analyzer(call_analyzer_id, analyzer_template_path=analyzer_template_path)
result = client.poll_result(response)
print("✅ Call recording analyzer created successfully!")

print(f"Analyzing call recording: {sample_file_path}")
print("⏳ Note: Audio analysis may take longer than document analysis...")
response = client.begin_analyze(call_analyzer_id, file_location=sample_file_path)
result_json = client.poll_result(response)

**Call Recording Analysis Results**

In [ ]:
print(json.dumps(result_json, indent=2))

**Clean Up Call Recording Analyzer**

Note: In production environments, analyzers are typically kept for reuse rather than deleted.

In [ ]:
client.delete_analyzer(call_analyzer_id)

## 5. Conversational Audio Analytics

Let's analyze the same audio file with a focus on conversational aspects like sentiment analysis and dialogue understanding.

Conversational audio analytics template:

In [ ]:
analyzer_template_path = '../analyzer_templates/conversational_audio_analytics.json'
with open(analyzer_template_path, 'r') as f:
    template_content = json.load(f)
    print(json.dumps(template_content, indent=2))

**Create and Run Conversational Audio Analyzer**

In [ ]:
sample_file_path = '../data/callCenterRecording.mp3'
conversation_analyzer_id = "conversational-audio-analytics-" + str(uuid.uuid4())

print(f"Creating conversational audio analyzer: {conversation_analyzer_id}")
response = client.begin_create_analyzer(conversation_analyzer_id, analyzer_template_path=analyzer_template_path)
result = client.poll_result(response)
print("✅ Conversational audio analyzer created successfully!")

print(f"Analyzing conversational audio: {sample_file_path}")
print("⏳ Note: Audio analysis may take longer than document analysis...")
response = client.begin_analyze(conversation_analyzer_id, file_location=sample_file_path)
result_json = client.poll_result(response)

**Conversational Audio Analysis Results**

In [ ]:
print(json.dumps(result_json, indent=2))

**Clean Up Conversational Audio Analyzer**

Note: In production environments, analyzers are typically kept for reuse rather than deleted.

In [ ]:
client.delete_analyzer(conversation_analyzer_id)

# Video Analysis

Now, let's explore video analysis capabilities. This modality can extract insights from video content including descriptions, sentiment analysis, and identification of key scenes.

## 6. Marketing Video Analysis

Let's analyze a marketing video to extract descriptions, sentiment, and key insights that could be valuable for content understanding and marketing analytics.

Content Understanding offers three segmentation options to slice a video, enabling you to get output for entire videos or short clips. You can specify these by setting the `segmentationMode` property in your custom analyzer:
- Whole-video – `"segmentationMode": "noSegmentation"`  
  The service treats the entire video file as a single segment and extracts metadata across its full duration.  
  Examples:
    - Compliance checks looking for specific brand-safety issues anywhere in an ad
    - Full-length descriptive summaries
- Automatic segmentation – `"segmentationMode": "auto"`  
  The service analyzes the timeline, automatically segmenting the video. It groups successive shots into coherent scenes, capped at one minute each.  
  Examples:
    - Creating storyboards from a show
    - Inserting mid-roll ads at logical pauses
- Custom segmentation – `"segmentationMode": "custom"`  
  You describe the segmentation logic in natural language, and the model creates segments accordingly. Set `segmentationDefinition` with a string describing your desired segmentation. Custom segmentation allows segments of varying length from seconds to minutes.  
  Example:
    - Breaking a news broadcast into distinct stories

### 6-1 Analyze Without Segmentation

In this example, we analyze a marketing video without segmentation.
- Please set `segmentationMode` to `noSegmentation` in the analyzer schema `config` to process the entire video as a single segment.

In [ ]:
analyzer_template_path = '../analyzer_templates/marketing_video.json'
with open(analyzer_template_path, 'r') as f:
    template_content = json.load(f)
    print(json.dumps(template_content, indent=2))

**Create and Run Marketing Video Analyzer**

In [ ]:
sample_file_path = '../data/FlightSimulator.mp4'
video_analyzer_id = "marketing-video-analytics-" + str(uuid.uuid4())

print(f"Creating marketing video analyzer: {video_analyzer_id}")
response = client.begin_create_analyzer(video_analyzer_id, analyzer_template_path=analyzer_template_path)
result = client.poll_result(response)
print("✅ Marketing video analyzer created successfully!")

print(f"Analyzing marketing video: {sample_file_path}")
print("⏳ Note: Video analysis may take significantly longer than document analysis...")
response = client.begin_analyze(video_analyzer_id, file_location=sample_file_path)
result_json = client.poll_result(response)

Marketing video analysis result:
- The result is generated from the content of the entire video.

In [ ]:
print(json.dumps(result_json, indent=2))

**Clean Up Marketing Video Analyzer**

Note: In production environments, analyzers are typically kept for reuse rather than deleting.

In [ ]:
client.delete_analyzer(video_analyzer_id)

### 6-2 Analyze With Automatic Segmentation

In this example, we use automatic segmentation for marketing video analytics.  
- Please set `segmentationMode` to `auto` in the analyzer schema `config` to enable automatic segmentation.
- The defined fields for segment analysis must be declared under `fieldSchema` → `Segments`.

In [ ]:
analyzer_template_path = '../analyzer_templates/marketing_video_segmenation_auto.json'
with open(analyzer_template_path, 'r') as f:
    template_content = json.load(f)
    print(json.dumps(template_content, indent=2))

**Create and Run Marketing Video Analyzer with Automatic Segmentation**

In [ ]:
sample_file_path = '../data/FlightSimulator.mp4'
video_analyzer_id = "marketing-video-analytics-" + str(uuid.uuid4())

print(f"Creating marketing video analyzer: {video_analyzer_id}")
response = client.begin_create_analyzer(video_analyzer_id, analyzer_template_path=analyzer_template_path)
result = client.poll_result(response)
print("✅ Marketing video analyzer created successfully!")

print(f"Analyzing marketing video: {sample_file_path}")
print("⏳ Note: Video analysis may take significantly longer than document analysis...")
response = client.begin_analyze(video_analyzer_id, file_location=sample_file_path)
result_json = client.poll_result(response)

Marketing video analysis result:
- The output includes automatically segmented clips with descriptions in the markdown content.  
- The analyzer generates the fields defined in the schema separately for each segment.

In [ ]:
print(json.dumps(result_json, indent=2))

**Clean Up Marketing Video Analyzer**

Note: In production environments, analyzers are typically kept for reuse rather than deleting.

In [ ]:
client.delete_analyzer(video_analyzer_id)

### 6-3 Analyze With Custom Segmentation

In this example, we use custom segmentation for marketing video analytics.  
- Please set `segmentationMode` to `custom`.  
- Provide a `segmentationDefinition` string describing how you'd like the video to be segmented.
- The defined fields for segment analysis must be declared under `fieldSchema` → `Segments`.

In [ ]:
analyzer_template_path = '../analyzer_templates/marketing_video_segmenation_custom.json'
with open(analyzer_template_path, 'r') as f:
    template_content = json.load(f)
    print(json.dumps(template_content, indent=2))

**Create and Run Marketing Video Analyzer with Custom Segmentation**

In [ ]:
sample_file_path = '../data/FlightSimulator.mp4'
video_analyzer_id = "marketing-video-analytics-" + str(uuid.uuid4())

print(f"Creating marketing video analyzer: {video_analyzer_id}")
response = client.begin_create_analyzer(video_analyzer_id, analyzer_template_path=analyzer_template_path)
result = client.poll_result(response)
print("✅ Marketing video analyzer created successfully!")

print(f"Analyzing marketing video: {sample_file_path}")
print("⏳ Note: Video analysis may take significantly longer than document analysis...")
response = client.begin_analyze(video_analyzer_id, file_location=sample_file_path)
result_json = client.poll_result(response)

Marketing video analysis result:
- The video is segmented according to your custom definition, with segment descriptions included in the markdown content.  
- The segmentation may differ from automatic segmentation results.  
- The analyzer generates the fields defined in the schema separately for each segment.

In [ ]:
print(json.dumps(result_json, indent=2))

**Clean Up Marketing Video Analyzer**

Note: In production environments, analyzers are typically kept for reuse rather than deleting.

In [ ]:
client.delete_analyzer(video_analyzer_id)

# Image Analysis

Finally, let's explore image analysis capabilities. This modality can extract information from charts, diagrams, and other visual content.

## 7. Chart and Image Analysis

Let's analyze a chart image to extract data points, trends, and insights from visual data representations.

In [ ]:
# Image chart analytics template
analyzer_template_path = '../analyzer_templates/image_chart.json'
with open(analyzer_template_path, 'r') as f:
    template_content = json.load(f)
    print(json.dumps(template_content, indent=2))

**Create and Run Chart Image Analyzer**

In [ ]:
sample_file_path = '../data/pieChart.jpg'
chart_analyzer_id = "chart-analysis-" + str(uuid.uuid4())

print(f"Creating chart analyzer: {chart_analyzer_id}")
response = client.begin_create_analyzer(chart_analyzer_id, analyzer_template_path=analyzer_template_path)
result = client.poll_result(response)
print("✅ Chart analyzer created successfully!")

print(f"Analyzing chart: {sample_file_path}")
response = client.begin_analyze(chart_analyzer_id, file_location=sample_file_path)
result_json = client.poll_result(response)

**Chart Analysis Result**

In [ ]:
print(json.dumps(result_json, indent=2))

**Clean Up Chart Image Analyzer**

Note: In production environments, analyzers are typically kept for reuse rather than deleting.

In [ ]:
client.delete_analyzer(chart_analyzer_id)

# Summary

🎉 **Congratulations!** You have successfully explored all the major modalities of Azure AI Content Understanding:

✅ **Document Analysis**: Extracted fields from invoices and receipts  
✅ **Audio Analysis**: Analyzed call recordings and conversational audio  
✅ **Video Analysis**: Processed marketing videos for insights  
✅ **Image Analysis**: Extracted information from charts and visual content

## Key Takeaways

- **Multi-modal capabilities**: Content Understanding processes documents, audio, video, and images
- **Customizable templates**: Each analyzer template is designed for specific use cases but can be customized
- **Automatic cleanup**: Each analyzer was cleaned up after use to manage resources
- **Structured output**: All results are returned in a consistent JSON format for easy integration

## Next Steps

- Explore the analyzer templates located in the `../analyzer_templates/` directory
- Modify existing templates or create custom ones tailored to your specific use cases
- Check out other notebooks in this repository for advanced scenarios
- Visit the [Azure AI Content Understanding documentation](https://docs.microsoft.com/azure/ai-services/content-understanding/) for more information